<a href="https://colab.research.google.com/github/mrudulamadhavan/My-Projects/blob/main/Fitness_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Vedaz Assignment - ML Intern**

## **Objective:**
Design and develop a basic agentic AI that can perform decision-making tasks based on a given input. The agent should simulate autonomy by taking actions based on predefined goals and dynamic environment factors.


### **1. Problem Statement:**
You are tasked with creating a simple agentic AI capable of learning from interactions within a given environment. This AI should be able to:

* Receive input data (e.g., a set of actions or conditions).
* Learn from these inputs (using basic reinforcement learning techniques, for example).
* Make decisions based on those learned behaviors to achieve predefined goals.



# **Agentic Fitness Chatbot with Real-Time Adaptation**

### **Project Overview**

The Agentic Fitness Chatbot is an intelligent assistant designed to help users achieve their fitness goals by providing personalized workout and meal recommendations. The bot adapts in real-time based on user feedback, continuously improving its suggestions by learning from the user's preferences and behavior. This project implements Q-learning, a reinforcement learning algorithm, to provide real-time adaptation to the chatbot's recommendations.

### **Features**
* User Profile Creation: Users can input their personal details (age, weight, height) to personalize the chatbot's recommendations.

* Fitness Goal Selection: Users can specify their fitness goal (weight loss, muscle gain, endurance) to receive tailored workout and meal suggestions.

* Real-Time Adaptation: The chatbot learns from the feedback provided by the user (e.g., ratings for workouts and meals) and improves its future suggestions using Q-learning.

* Progress Tracking: Users' workout feedback (ratings and like/dislike) helps the bot to adjust future recommendations.

### **Technologies Used**
* Python: Main programming language for developing the chatbot and implementing Q-learning.

* Streamlit: For building the interactive web interface.

* Pandas and Numpy: For managing data and mathematical operations.



In [ ]:
# importing libraries
import streamlit as st
import pandas as pd
import numpy as np
import random

In [ ]:
# Sample Data for workouts and meals
workouts_df = pd.DataFrame({
    'exercise': ['Running', 'Cycling', 'Yoga', 'Swimming', 'Deadlifts', 'Push-ups'],
    'calories_per_hour': [600, 500, 200, 550, 350, 400],
    'workout_type': ['Cardio', 'Cardio', 'Flexibility', 'Cardio', 'Strength', 'Strength']
})

meals_df = pd.DataFrame({
    'meal_name': ['Oatmeal', 'Chicken Salad', 'Pasta', 'Smoothie', 'Salmon', 'Eggs'],
    'calories': [150, 400, 500, 250, 600, 300],
    'meal_type': ['Breakfast', 'Lunch', 'Dinner', 'Snack', 'Dinner', 'Breakfast']
})

In [ ]:
# Q-Learning Parameters
ALPHA = 0.1  # Learning rate
GAMMA = 0.9  # Discount factor
EPSILON = 0.1  # Exploration rate (probability to explore instead of exploiting the learned policy)

class FitnessAgent:
    def __init__(self):
        self.workout_feedback = {}  # Stores feedback for workouts
        self.meal_feedback = {}  # Stores feedback for meals
        self.q_table_workouts = np.zeros((3, 6))  # 3 states (goals), 6 actions (workouts)
        self.q_table_meals = np.zeros((3, 6))  # 3 states (goals), 6 actions (meals)
        self.states = ['weight_loss', 'muscle_gain', 'endurance']

    def get_state_index(self, goal):
        """Convert the goal string to an index"""
        return self.states.index(goal)

    def recommend_workout(self, goal):
        """Recommend a workout based on the Q-table"""
        state_index = self.get_state_index(goal)

        if random.uniform(0, 1) < EPSILON:  # Exploration: pick a random action
            action = random.choice(range(6))
        else:  # Exploitation: pick the best action based on Q-table
            action = np.argmax(self.q_table_workouts[state_index])

        recommended_workout = workouts_df.iloc[action]
        return recommended_workout, action

    def recommend_meal(self, goal):
        """Recommend a meal based on the Q-table"""
        state_index = self.get_state_index(goal)

        if random.uniform(0, 1) < EPSILON:  # Exploration: pick a random action
            action = random.choice(range(6))
        else:  # Exploitation: pick the best action based on Q-table
            action = np.argmax(self.q_table_meals[state_index])

        recommended_meal = meals_df.iloc[action]
        return recommended_meal, action

    def collect_feedback(self, item_type, item_name, feedback, rating=None):
        """Learn from feedback (positive/negative or rating)"""
        reward = rating if rating else (5 if feedback == 'like' else 1)

        if item_type == 'workout':
            workout_index = workouts_df[workouts_df['exercise'] == item_name].index[0]
            state_index = self.get_state_index(feedback)
            self.q_table_workouts[state_index, workout_index] = (1 - ALPHA) * self.q_table_workouts[state_index, workout_index] + ALPHA * (reward + GAMMA * np.max(self.q_table_workouts[state_index]))

        elif item_type == 'meal':
            meal_index = meals_df[meals_df['meal_name'] == item_name].index[0]
            state_index = self.get_state_index(feedback)
            self.q_table_meals[state_index, meal_index] = (1 - ALPHA) * self.q_table_meals[state_index, meal_index] + ALPHA * (reward + GAMMA * np.max(self.q_table_meals[state_index]))

    def track_progress(self, workout, calories_burned, meals_consumed):
        """Track the user's progress"""
        pass  # For simplicity, we'll skip progress tracking here

# Initialize the fitness agent
fitness_agent = FitnessAgent()

In [ ]:
# Streamlit interface
st.title("Agentic Fitness Chatbot with Real-Time Adaptation")

# User Profile
st.header("Create Your Profile")

if 'profile_created' not in st.session_state:
    st.session_state.profile_created = False

if not st.session_state.profile_created:
    # Profile Creation Form
    with st.form(key='profile_form'):
        name = st.text_input("Name")
        age = st.number_input("Age", min_value=18, max_value=100)
        weight = st.number_input("Weight (kg)", min_value=30, max_value=200)
        height = st.number_input("Height (cm)", min_value=100, max_value=250)
        profile_submit = st.form_submit_button("Create Profile")

        if profile_submit:
            fitness_agent.create_profile(name, age, weight, height)
            st.session_state.profile_created = True
            st.success("Profile Created Successfully!")

else:
    # Display profile information
    st.write(f"**Name**: {fitness_agent.user_profile['name']}")
    st.write(f"**Age**: {fitness_agent.user_profile['age']}")
    st.write(f"**Weight**: {fitness_agent.user_profile['weight']} kg")
    st.write(f"**Height**: {fitness_agent.user_profile['height']} cm")

    # Fitness Goal Input
    goal = st.selectbox("What is your fitness goal?", ["Select", "weight_loss", "muscle_gain", "endurance"])

    if goal != "Select":
        st.write(f"Your goal is {goal.capitalize()}!")

        # Get workout recommendations
        recommended_workout, workout_action = fitness_agent.recommend_workout(goal)
        st.subheader("Recommended Workout")
        st.write(f"**{recommended_workout['exercise']}** - {recommended_workout['calories_per_hour']} calories per hour")

        # Collect feedback on workout (Rating)
        workout_feedback = st.radio("Did you like this workout?", ['like', 'dislike'], key="workout_feedback")
        workout_rating = st.slider("Rate this workout (1-5)", 1, 5, key="workout_rating")
        if st.button("Submit Workout Feedback"):
            fitness_agent.collect_feedback('workout', recommended_workout['exercise'], workout_feedback, workout_rating)

        # Get meal recommendations
        recommended_meal, meal_action = fitness_agent.recommend_meal(goal)
        st.subheader("Recommended Meal")
        st.write(f"**{recommended_meal['meal_name']}** - {recommended_meal['calories']} calories")

        # Collect feedback on meal (Rating)
        meal_feedback = st.radio("Did you like this meal?", ['like', 'dislike'], key="meal_feedback")
        meal_rating = st.slider("Rate this meal (1-5)", 1, 5, key="meal_rating")
        if st.button("Submit Meal Feedback"):
            fitness_agent.collect_feedback('meal', recommended_meal['meal_name'], meal_feedback, meal_rating)

        # Show updated recommendations based on real-time adaptation
        if st.button("Show Adjusted Recommendations"):
            st.write("Updated recommendations based on your feedback:")
            new_workout, _ = fitness_agent.recommend_workout(goal)
            st.write(f"**{new_workout['exercise']}** - {new_workout['calories_per_hour']} calories per hour")

            new_meal, _ = fitness_agent.recommend_meal(goal)
            st.write(f"**{new_meal['meal_name']}** - {new_meal['calories']} calories")

    else:
        st.write("Please select a fitness goal to get started!")